### 1. Introduction

Solving this assignment will give you an idea about how real business problems are solved using EDA. In this case study, apart from applying the techniques you have learnt in EDA, you will also develop a basic understanding of risk analytics in banking and financial services and understand how data is used to minimise the risk of losing money while lending to customers.


### 2. Business Understanding

1. You work for a consumer finance company which specialises in lending various types of loans to urban customers. When the company receives a loan application, the company has to make a decision for loan approval based on the applicant’s profile. Two types of risks are associated with the bank’s decision:
    1. If the applicant is likely to repay the loan, then not approving the loan results in a loss of business to the company
    2. If the applicant is not likely to repay the loan, i.e. he/she is likely to default, then approving the loan may lead to a financial loss for the company

1. The data given below contains information about past loan applicants and whether they ‘defaulted’ or not. **The aim is to identify patterns which indicate if a person is likely to default, which may be used for taking actions such as denying the loan, reducing the amount of loan, lending (to risky applicants) at a higher interest rate, etc.**

1. In this case study, you will use EDA to understand how consumer attributes and loan attributes influence the tendency of default.

1. When a person applies for a loan, there are two types of decisions that could be taken by the company:
    1. **Loan accepted:** If the company approves the loan, there are 3 possible scenarios described below:
        1. **Fully paid:** Applicant has fully paid the loan (the principal and the interest rate)
        1. **Current:** Applicant is in the process of paying the instalments, i.e. the tenure of the loan is not yet completed. These candidates are not labelled as 'defaulted'.
        1. **Charged-off:** Applicant has not paid the instalments in due time for a long period of time, i.e. he/she has defaulted on the loan 
    1. **Loan rejected:** The company had rejected the loan (because the candidate does not meet their requirements etc.). Since the loan was rejected, there is no transactional history of those applicants with the company and so this data is not available with the company (and thus in this dataset)

 
### 3. Business Objectives

1. This company is the largest online loan marketplace, facilitating personal loans, business loans, and financing of medical procedures. Borrowers can easily access lower interest rate loans through a fast online interface. 

1. Like most other lending companies, lending loans to ‘risky’ applicants is the largest source of financial loss (called credit loss). Credit loss is the amount of money lost by the lender when the borrower refuses to pay or runs away with the money owed. In other words, borrowers who default cause the largest amount of loss to the lenders. In this case, the customers labelled as 'charged-off' are the 'defaulters'. 

1. If one is able to identify these risky loan applicants, then such loans can be reduced thereby cutting down the amount of credit loss. **Identification of such applicants using EDA is the aim of this case study.**

1. In other words, the company wants to understand **the driving factors (or driver variables) behind loan default**, i.e. the variables which are strong indicators of default.  The company can utilise this knowledge for its portfolio and risk assessment. 

1. To develop your understanding of the domain, you are advised to independently research a little about risk analytics (understanding the types of variables and their significance should be enough).

 
### 4. Results Expected
1. Write all your code in one well-commented Python file; briefly mention the insights and observations from the analysis 
1. Present the overall approach of the analysis in a presentation: 
    1. Mention the problem statement and the analysis approach briefly 
    1. Explain the results of univariate, bivariate analysis etc. in business terms
    1. Include visualisations and summarise the most important results in the presentation

### 5. Importing libraries

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import statsmodels
import sklearn


### 6. Reading dataset

In [27]:
# Load the Dataset CSV file
lendingCaseStudyDataFrame = pd.read_csv('loan.csv')

# Load the metadata file
lendingCaseStudyMetadata = pd.read_excel('Data_Dictionary.xlsx', sheet_name='LoanStats')

/var/folders/kl/lhs7mp5s1ml8g684db055ckm0000gq/T/ipykernel_49095/1161814571.py:2: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  lendingCaseStudyDataFrame = pd.read_csv('loan.csv')


### 7. DATA CLEANING ACTIONS


#### 7.1 FIXING ROWS AND COLUMNS


##### 7.1.1 Fixing columns
1. Some of the columns are unncessary as it does not impact the results i.e. loan descisions. It doesn't provide any variability or meaningful information that could contribute to analysis. Keeping such columns could introduce noise or redundancy, so dropping them makes your dataset cleaner and more efficient to work with.
1. Some of the columns have mixed types as described by the error when read by pandas.



In [28]:
# Listing types and unique values of each column
pd.reset_option('display.max_seq_items')
print("Columns and column types in the CSV file:")
for columnName in lendingCaseStudyDataFrame.columns:
    columnType = lendingCaseStudyDataFrame[columnName].apply(type).unique()
    uniqueColumnValues = lendingCaseStudyDataFrame[columnName].unique()
    print(f"The data type of {columnName} is: {columnType}")
    print(f"The number of unique values of {columnName} is: {uniqueColumnValues.size}")

    # List all column values if size < 10; To investigate if the column does not have any value of need.
    if (uniqueColumnValues.size < 10):
        print(f"The unique values of {columnName} is: {uniqueColumnValues}")
    print('\n')


Columns and column types in the CSV file:
The data type of id is: [<class 'int'>]
The number of unique values of id is: 39717


The data type of member_id is: [<class 'int'>]
The number of unique values of member_id is: 39717


The data type of loan_amnt is: [<class 'int'>]
The number of unique values of loan_amnt is: 885


The data type of funded_amnt is: [<class 'int'>]
The number of unique values of funded_amnt is: 1041


The data type of funded_amnt_inv is: [<class 'float'>]
The number of unique values of funded_amnt_inv is: 8205


The data type of term is: [<class 'str'>]
The number of unique values of term is: 2
The unique values of term is: [' 36 months' ' 60 months']


The data type of int_rate is: [<class 'str'>]
The number of unique values of int_rate is: 371


The data type of installment is: [<class 'float'>]
The number of unique values of installment is: 15383


The data type of grade is: [<class 'str'>]
The number of unique values of grade is: 7
The unique values of grade

In [29]:
# Dropping 'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'tot_coll_amt', 'open_acc_6m',
# 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
# 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 
# 'bc_util', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc',
# 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 
# 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 
# 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'tot_hi_cred_lim',
# 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit', 'tot_cur_bal'
# as all values are 'nan'
columnsListWithAllValuesNan = ['mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'tot_coll_amt', 'open_acc_6m',
'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 
'bc_util', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc',
'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 
'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 
'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'tot_hi_cred_lim',
'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit', 'tot_cur_bal']

# Dropping 'policy_code' column as all values are 1.
# Dropping 'application_type' column as all values are individual
# Dropping 'acc_now_delinq', 'delinq_amnt', column as all values are 0
# Dropping 'pymnt_plan' column as all values are 'n'.
# Dropping 'initial_list_status' as all values are 'f'.
columnsListWithAllSameValues = ['policy_code', 'application_type', 'acc_now_delinq', 'delinq_amnt', 'pymnt_plan', 'initial_list_status']

# Dropping 'collections_12_mths_ex_med', 'chargeoff_within_12_mths', 'tax_liens' as all values are either 'nan or 0'
columnsListWithEitherNanOrOs = ['collections_12_mths_ex_med', 'chargeoff_within_12_mths', 'tax_liens', 'delinq_amnt']

# Dropping irrelvant columns not useful for analysis
columnsWithNoRelevancyForResults = ['url']

lendingCaseStudyDataFrameCleaned = lendingCaseStudyDataFrame.drop(columns=columnsListWithAllValuesNan + columnsListWithAllSameValues + columnsListWithEitherNanOrOs
                                                                 + columnsWithNoRelevancyForResults)

lendingCaseStudyDataFrameCleaned.head()




,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,pub_rec_bankruptcies
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,5000.00,863.16,0.00,0.00,0.00,Jan-15,171.62,NaN,May-16,0.0
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.0
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,2400.00,605.67,0.00,0.00,0.00,Jun-14,649.91,NaN,May-16,0.0
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,10000.00,2214.92,16.97,0.00,0.00,Jan-15,357.48,NaN,Apr-16,0.0
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,2475.94,1037.39,0.00,0.00,0.00,May-16,67.79,Jun-16,May-16,0.0


In [30]:
# Printing the List all column names individually and its type again after removal of non-important and empty columns.
print("Info and types of datasets after dropping unncessary columns: ")
print(lendingCaseStudyDataFrameCleaned.info())
print("\n")
print("Columns and column types in the Cleaned CSV file:")
for columnName in lendingCaseStudyDataFrameCleaned.columns:
    columnType = lendingCaseStudyDataFrameCleaned[columnName].apply(type).unique()
    uniqueColumnValues = lendingCaseStudyDataFrameCleaned[columnName].unique()
    print(f"The data type of {columnName} is: {columnType}")
    print(f"The number of unique values of {columnName} is: {uniqueColumnValues.size}")

    # List all column values if size < 10; To investigate if the column does not have any value of need.
    if (uniqueColumnValues.size < 10):
        print(f"The unique values of {columnName} is: {uniqueColumnValues}")
    print('\n')


Info and types of datasets after dropping unncessary columns: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       39717 non-null  int64  
 1   member_id                39717 non-null  int64  
 2   loan_amnt                39717 non-null  int64  
 3   funded_amnt              39717 non-null  int64  
 4   funded_amnt_inv          39717 non-null  float64
 5   term                     39717 non-null  object 
 6   int_rate                 39717 non-null  object 
 7   installment              39717 non-null  float64
 8   grade                    39717 non-null  object 
 9   sub_grade                39717 non-null  object 
 10  emp_title                37258 non-null  object 
 11  emp_length               38642 non-null  object 
 12  home_ownership           39717 non-null  object 
 13  annual_inc   

In [31]:
# Fixing data types of each column.

from datetime import datetime
from enum import Enum
import math

class HomeOwnership(Enum):
    RENT = "RENT"
    OWN = "OWN"
    MORTGAGE = "MORTGAGE"
    NONE = "NONE"
    OTHER = "OTHER"

class VerificationStatus(Enum):
    VERIFIED = "Verified"
    SOURCE_VERIFIED = "Source Verified"
    NOT_VERIFIED = "Not Verified"
    
class LoanStatus(Enum):
    CURRENT = "Current"
    FULLY_PAID = 'Fully Paid'
    CHARGED_OFF = 'Charged Off'

class Grade(Enum):
    A = "A"
    B = "B"
    C = "C"
    D = "D"
    E = "E"
    F = "F"
    G = "G"

# Define conversion functions
def toString(value):
    return str(value)

def toFloat(value):
    if isinstance(value, float) and math.isnan(value):
        return None  # Return the None.
    elif isinstance(value, str) or isinstance(value, int) or isinstance(value, float):
        return float(value)
    else:
        raise ValueError(f"Unexpected value type: {type(value)}")

def toInt(value):
    if isinstance(value, float) and math.isnan(value):
        return None  # Return None.
    elif isinstance(value, str) or isinstance(value, int) or isinstance(value, float):
        return int(value)
    else:
        raise ValueError(f"Unexpected value type: {type(value)}")

    
def termToInt(value):
    if isinstance(value, float) and math.isnan(value):
        return None  # Return None.
    elif isinstance(value, str):
        value = value.rstrip(' months')
        return int(value)
    else:
        raise ValueError(f"Unexpected value type: {type(value)}")

def interestToFloat(value):
    if isinstance(value, float) and math.isnan(value):
        return None  # Return None.
    elif isinstance(value, str):
        value = value.strip()  # Remove any leading/trailing spaces
        if value.endswith('%'):
            value = value.rstrip('%')
        try:
            return float(value) / 100.0
        except ValueError:
            raise ValueError(f"Cannot convert '{value}' to a float.")
    else:
        raise ValueError(f"Unexpected value type: {type(value)}")


def gradeToGradeType(value):
    value = value.rstrip('%')
    return float(value)/100.0;

def toGrade(gradeStr: str) -> Grade:
    for grade in Grade:
        if grade.value == gradeStr:
            return grade
    raise ValueError(f"'{gradeStr}' is not a valid Grade")

def toVerificationStatus(verificationStatusStr: str) -> VerificationStatus:
    for status in VerificationStatus:
        if status.value == verificationStatusStr:
            return status
    raise ValueError(f"'{verificationStatusStr}' is not a valid VerificationStatus")

def toDatetimeParsingMonthYear(dateTimeStr) -> datetime:
    if dateTimeStr is None or (isinstance(dateTimeStr, float) and math.isnan(dateTimeStr)):
        return None  # return None.
    if not isinstance(dateTimeStr, str):
        raise TypeError(f"Expected a string for dateTimeStr, but got {type(dateTimeStr).__name__}")
    try:
        return datetime.strptime(dateTimeStr, "%b-%y")
    except ValueError as e:
        raise ValueError(f"Error parsing date string '{dateTimeStr}': {e}")

def toHomeOwnership(homeOwnershipStr: str) -> HomeOwnership:
    for ownership in HomeOwnership:
        if ownership.value == homeOwnershipStr:
            return ownership
    raise ValueError(f"'{homeOwnershipStr}' is not a valid HomeOwnership")


def toLoanStatus(loanStatusStr: str) -> LoanStatus:
    for status in LoanStatus:
        if status.value == loanStatusStr:
            return status
    raise ValueError(f"'{loanStatusStr}' is not a valid LoanStatus")

# Define a conversion map (like a case switch)
conversionMap = {
    'id': toString,    # Convert 'id' column to string
    'member_id': toString, # Convert 'member_id' column to string
    'loan_amnt': toFloat, # Convert 'loan_amnt' column to float.
    'funded_amnt': toFloat,  # Convert 'funded_amnt' column to float.
    'funded_amnt_inv' : toFloat, # Convert 'funded_amnt_inv' column to float.
    'term' : termToInt, # Convert 'term' column to int to enable calculations.
    'int_rate' : interestToFloat, # Convert 'int_rate' column to float to enable calculations.
    'installment' : toFloat, # Convert 'installment' column to float to enable calculations though it already is.
    'grade' : toGrade, # Convert 'grade' column to Grade enum.
    'sub_grade' : toString, # Convert 'sub_grade' column to string though it already is.
    'emp_title' : toString, # Convert 'emp_title' column to string though it already is.
    'emp_length' : toString, # Convert 'emp_length' column to string though it already is.
    'home_ownership': toHomeOwnership, # Convert 'home_ownership' column to HomeOwnership enum. 
    'annual_inc' : toFloat, # Convert 'annual_inc' column to float to enable calculations though it already is.
    'verification_status' : toVerificationStatus, # Convert 'verification_status' column to VerificationStatus enum. 
    'issue_d' : toDatetimeParsingMonthYear, # Convert 'issue_d' column to DateTimeStamp type.
    'loan_status' : toLoanStatus, # Convert 'loan_status' column to LoanStatus enum.
    'url' : toString, # Convert 'url' column to string though it already is.
    'desc' : toString, # Convert 'desc' column to string though it already is.
    'purpose' : toString, # Convert 'purpose' column to string though it already is.
    'title' : toString, # Convert 'title' column to string.
    'zip_code' : toString, # Convert 'zip_code' column to string though it already is.
    'addr_state' : toString, # Convert 'addr_state' column to string though it already is.
    'dti' : toFloat, # Convert 'dti' column to float though it already is.
    'delinq_2yrs' : toInt, # Convert 'delinq_2yrs' column to int though it already is.
    'earliest_cr_line' : toDatetimeParsingMonthYear, # Convert 'earliest_cr_line' column to DateTimeStamp type.
    'inq_last_6mths' : toInt, # Convert 'inq_last_6mths' column to int though it already is.
    'mths_since_last_delinq' : toInt, # Convert 'mths_since_last_delinq' column to int.
    'mths_since_last_record' : toInt, # Convert 'mths_since_last_record' column to int.
    'open_acc' : toInt, # Convert 'open_acc' column to int.
    'pub_rec' : toInt, # Convert 'pub_rec' column to int.
    'revol_bal' : toFloat, # Convert 'revol_bal' column to float.
    'revol_util' : interestToFloat, # Convert 'revol_util' column to float.
    'total_acc' : toInt, # Convert 'total_acc' column to int though it already is.
    'out_prncp' : toFloat, # Convert 'out_prncp' column to float though it already is.
    'out_prncp_inv' : toFloat, # Convert 'out_prncp_inv' column to float though it already is.
    'total_pymnt' : toFloat, # Convert 'total_pymnt' column to float though it already is.
    'total_pymnt_inv' : toFloat, # Convert 'total_pymnt_inv' column to float though it already is.
    'total_rec_prncp' : toFloat, # Convert 'total_rec_prncp' column to float though it already is.
    'total_rec_int' : toFloat, # Convert 'total_rec_int' column to float though it already is.
    'total_rec_late_fee' : toFloat, # Convert 'total_rec_late_fee' column to float though it already is.
    'recoveries' : toFloat, # Convert 'recoveries' column to float though it already is.
    'collection_recovery_fee' : toFloat, # Convert 'collection_recovery_fee' column to float though it already is.
    'last_pymnt_d' : toDatetimeParsingMonthYear, # Convert 'last_pymnt_d' column to DateTimeStamp type.
    'last_pymnt_amnt' : toFloat, # Convert 'last_pymnt_amnt' column to float though it already is.
    'next_pymnt_d' : toDatetimeParsingMonthYear, # Convert 'next_pymnt_d' column to DateTimeStamp type.
    'last_credit_pull_d' : toDatetimeParsingMonthYear, # Convert 'last_credit_pull_d' column to DateTimeStamp type.
    'pub_rec_bankruptcies' : toInt, # Convert 'pub_rec_bankruptcies' column to int type.
}

lendingCaseStudyDataFrameCleanedWithTypesCorrected = pd.DataFrame()
for columnName in lendingCaseStudyDataFrameCleaned.columns:
    lendingCaseStudyDataFrameCleanedWithTypesCorrected[columnName] = lendingCaseStudyDataFrameCleaned[columnName].apply(conversionMap[columnName])

In [32]:
# Printing the List all column names individually and its type again after correcting the type of columns.

print("Info and types of datasets after correcting the types of columns: ")
print(lendingCaseStudyDataFrameCleanedWithTypesCorrected.info())
print("\n")
print("Columns and column types in the Type corrected CSV file:")
for columnName in lendingCaseStudyDataFrameCleanedWithTypesCorrected.columns:
    columnType = lendingCaseStudyDataFrameCleanedWithTypesCorrected[columnName].apply(type).unique()
    uniqueColumnValues = lendingCaseStudyDataFrameCleanedWithTypesCorrected[columnName].unique()
    print(f"The data type of {columnName} is: {columnType}")
    print(f"The number of unique values of {columnName} is: {uniqueColumnValues.size}")

    # List all column values if size < 10; To investigate if the column does not have any value of need.
    if (uniqueColumnValues.size < 10):
        print(f"The unique values of {columnName} is: {uniqueColumnValues}")
    print('\n')


Info and types of datasets after correcting the types of columns: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       39717 non-null  object        
 1   member_id                39717 non-null  object        
 2   loan_amnt                39717 non-null  float64       
 3   funded_amnt              39717 non-null  float64       
 4   funded_amnt_inv          39717 non-null  float64       
 5   term                     39717 non-null  int64         
 6   int_rate                 39717 non-null  float64       
 7   installment              39717 non-null  float64       
 8   grade                    39717 non-null  object        
 9   sub_grade                39717 non-null  object        
 10  emp_title                39717 non-null  object        
 11  emp_length               3

In [33]:
# Fixing missing values whereever required.
lendingCaseStudyDataFrameCleanedWithTypesCorrected.isnull().sum()


id                             0
member_id                      0
loan_amnt                      0
funded_amnt                    0
funded_amnt_inv                0
term                           0
int_rate                       0
installment                    0
grade                          0
sub_grade                      0
emp_title                      0
emp_length                     0
home_ownership                 0
annual_inc                     0
verification_status            0
issue_d                        0
loan_status                    0
desc                           0
purpose                        0
title                          0
zip_code                       0
addr_state                     0
dti                            0
delinq_2yrs                    0
earliest_cr_line               0
inq_last_6mths                 0
mths_since_last_delinq     25682
mths_since_last_record     36931
open_acc                       0
pub_rec                        0
revol_bal 

In [34]:
# Imputing missing values in 'months_since_last_delinquency', 'mths_since_last_record' with a large number, based on the assumption that missing data indicates no delinquency or public record has occurred.
# This approach allows for analysis by treating the missing data as an indication that the event never happened, enabling relationships with other variables to be explored.
lendingCaseStudyDataFrameCleanedWithTypesCorrected['mths_since_last_delinq'] = lendingCaseStudyDataFrameCleanedWithTypesCorrected['mths_since_last_delinq'].fillna(99999)
lendingCaseStudyDataFrameCleanedWithTypesCorrected['mths_since_last_record'] = lendingCaseStudyDataFrameCleanedWithTypesCorrected['mths_since_last_record'].fillna(99999)

In [35]:
# Columns with Null Values:
# revol_util : Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.
# last_pymnt_d : Last month payment was received
# next_pymnt_d: Next scheduled payment date
# last_credit_pull_d: The most recent month LC pulled credit for this loan
# pub_rec_bankruptcies: Number of public record bankruptcies 

# Following are actions need to be taken on the dataset assuming the following:
# 1. Revolving Line Utilization Rate (revol_util): The revol_util value is null because the borrower does not have a credit card. Given that only 50 out of 39,717 records have null values, it may be reasonable to exclude these records from the analysis.

# 2. last_pymnt_d is null: The last_pymnt_d value is null because the borrower has never made a payment to LendingClub. This assumption can be validated by examining the loan status of records where last_pymnt_d is null. Therefore, it is acceptable to retain these null values in the dataset.
print('----------------last_pymnt_d: loanStaus-------------')
print(lendingCaseStudyDataFrameCleanedWithTypesCorrected[lendingCaseStudyDataFrameCleanedWithTypesCorrected['last_pymnt_d'].isnull()]['loan_status'].unique())
print()
# 3. next_pymnt_d: The next_pymnt_d value is null because the loan has been terminated, meaning the loan status is either charged off or fully paid. Therefore, it is acceptable to retain these null values in the dataset.
print('----------------next_pymnt_d: loanStaus-------------')
print(lendingCaseStudyDataFrameCleanedWithTypesCorrected[lendingCaseStudyDataFrameCleanedWithTypesCorrected['next_pymnt_d'].isnull()]['loan_status'].unique())
print()
# 4. last_credit_pull_d: The last_credit_pull_d value is null because LendingClub never pulled the borrower's credit history. Given that there are only 2 such records, it may be reasonable to remove these rows from the dataset.

# 5. pub_rec_bankruptcies is null The pub_rec_bankruptcies value is null due to the unavailability of data from government sources. Since only 697 out of 39,717 records have null values, it may be reasonable to exclude these rows from the analysis.

----------------last_pymnt_d: loanStaus-------------
[<LoanStatus.CHARGED_OFF: 'Charged Off'>]

----------------next_pymnt_d: loanStaus-------------
[<LoanStatus.FULLY_PAID: 'Fully Paid'>
 <LoanStatus.CHARGED_OFF: 'Charged Off'>]



##### 7.1.2 Fixing Rows
1. Before we fix rows, let's look at the definition of each of the column.
```
loan_amnt --> The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.
funded_amnt --> The total amount committed to that loan at that point in time.
funded_amnt_inv --> The total amount committed by investors for that loan at that point in time.
term --> The number of payments on the loan. Values are in months and can be either 36 or 60.
int_rate --> Interest Rate on the loan
installment --> The monthly payment owed by the borrower if the loan originates.
issue_d --> The month which the loan was funded
delinq_2yrs --> The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the        past 2 years
earliest_cr_line --> The month the borrower's earliest reported credit line was opened
mths_since_last_delinq --> The number of months since the borrower's last delinquency.
mths_since_last_record --> The number of months since the last public record.
open_acc --> The number of open credit lines in the borrower's credit file.
pub_rec --> Number of derogatory public records
revol_bal --> Total credit revolving balance
revol_util --> Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.
total_acc --> The total number of credit lines currently in the borrower's credit file
out_prncp	--> Remaining outstanding principal for total amount funded
out_prncp_inv	--> Remaining outstanding principal for portion of total amount funded by investors
total_pymnt --> Payments received to date for total amount funded
total_pymnt_inv --> Payments received to date for portion of total amount funded by investors
total_rec_prncp --> Principal received to date
total_rec_int	--> Interest received to date
total_rec_late_fee --> Late fees received to date
recoveries --> post charge off gross recovery (only for charged off loans)
collection_recovery_fee --> post charge off collection fee; refers to the costs incurred by a lender or creditor       when attempting to recover a debt after it has been charged off. These fees are typically paid to third-party       collection agencies, legal firms, or other entities involved in the collection process
last_pymnt_d --> Last month payment was received
next_pymnt_d --> Next scheduled payment date
last_pymnt_amnt --> Last total payment amount received
revol_util --> Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.
last_credit_pull_d --> The most recent month LC pulled credit for this loan
pub_rec_bankruptcies --> Number of public record bankruptcies 
```
2. Removal of rows as described above:
   1. Revolving Line Utilization Rate (revol_util): The revol_util value is null because the borrower does not have a credit card. Given that only 50 out of 39,717 records have null values, it may be reasonable to exclude these records from the analysis.
   2. last_credit_pull_d: The last_credit_pull_d value is null because LendingClub never pulled the borrower's credit history. Given that there are only 2 such records, it may be reasonable to remove these rows from the dataset.
   3. pub_rec_bankruptcies is null The pub_rec_bankruptcies value is null due to the unavailability of data from government sources. Since only 697 out of 39,717 records have null values, it may be reasonable to exclude these rows from the analysis.
2. Ensure that basic validations amongst different columns hold true for all rows. This includes following:
   1. Loan Amount Consistency: loan_amnt > funded_amnt and  loan_amnt > funded_amnt_inv
   2. Validate the term is either 36 or 60 months
   3. Validate that last_payment_date < next_payment_date
   4. Validate that earliest_cr_line <= issue_d
   5. Validate that open_acc <= total_acc
   6. Validate that total_rec_pricpal <= loan_amnt
   7. Validate that installment amount matches from the form
   8. Validate that pub_rec > pub_rec_bankruptcies
   9. Total Payments Consistency with interest and prinicipal:
      1. total_pymnt + total_pymnt_inv == total_rec_prncp + total_rec_int + total_rec_late_fee + recoveries
      2. total_rec_prncp = out_prncp + out_prncp_inv
   10. Loan status, HomeOwnership coolumn values are already verified via the enum types.


In [36]:
# Drop rows where the 'revol_util', 'last_credit_pull_d', and 'pub_rec_bankruptcies' column is null
lendingCaseStudyDataFrameCleanedWithTypesCorrected = lendingCaseStudyDataFrameCleanedWithTypesCorrected.dropna(subset=['revol_util', 'last_credit_pull_d', 'pub_rec_bankruptcies'])

# Display the resulting DataFrame
print(f"Number of records after dropping rows with null 'revol_util', 'last_credit_pull_d' and 'pub_rec_bankruptcies': {len(lendingCaseStudyDataFrameCleanedWithTypesCorrected)}")
print(f"Number of records with null values against each column:")
lendingCaseStudyDataFrameCleanedWithTypesCorrected.isnull().sum()


Number of records after dropping rows with null 'revol_util', 'last_credit_pull_d' and 'pub_rec_bankruptcies': 38969
Number of records with null values against each column:


id                             0
member_id                      0
loan_amnt                      0
funded_amnt                    0
funded_amnt_inv                0
term                           0
int_rate                       0
installment                    0
grade                          0
sub_grade                      0
emp_title                      0
emp_length                     0
home_ownership                 0
annual_inc                     0
verification_status            0
issue_d                        0
loan_status                    0
desc                           0
purpose                        0
title                          0
zip_code                       0
addr_state                     0
dti                            0
delinq_2yrs                    0
earliest_cr_line               0
inq_last_6mths                 0
mths_since_last_delinq         0
mths_since_last_record         0
open_acc                       0
pub_rec                        0
revol_bal 

In [41]:
# validations amongst different columns for all rows

# 1. Validate Loan Amount Consistency
loan_amount_inconsistency1 = lendingCaseStudyDataFrameCleanedWithTypesCorrected[lendingCaseStudyDataFrameCleanedWithTypesCorrected['loan_amnt'] < lendingCaseStudyDataFrameCleanedWithTypesCorrected['funded_amnt'] ]
loan_amount_inconsistency2 = lendingCaseStudyDataFrameCleanedWithTypesCorrected[lendingCaseStudyDataFrameCleanedWithTypesCorrected['loan_amnt'] < lendingCaseStudyDataFrameCleanedWithTypesCorrected['funded_amnt_inv'] ]
print(f"1.1 Number of inconsistent rows for loan amount vs funded_amnt: {len(loan_amount_inconsistency1)}")
print(f"1.2 Number of inconsistent rows for loan amount vs funded_amnt_inv: {len(loan_amount_inconsistency2)}")
print()
# 2. Validate the term is either 36 or 60 months
term_inconsistency = lendingCaseStudyDataFrameCleanedWithTypesCorrected[(lendingCaseStudyDataFrameCleanedWithTypesCorrected['term'] != 36) & (lendingCaseStudyDataFrameCleanedWithTypesCorrected['term']!=60)]
print(f"2. Number of inconsistent rows for term_inconsistency: {len(term_inconsistency)}")
print()
# 3. Validate that last_payment_date <= next_payment_date
payment_d_inconsistency = lendingCaseStudyDataFrameCleanedWithTypesCorrected[lendingCaseStudyDataFrameCleanedWithTypesCorrected['last_pymnt_d'] > lendingCaseStudyDataFrameCleanedWithTypesCorrected['next_pymnt_d']]
print(f"3. Number of inconsistent rows for payment_d_inconsistency: {len(payment_d_inconsistency)}")
print()
# 4. Validate that earliest_cr_line <= issue_d
earliest_cr_inconsistency = lendingCaseStudyDataFrameCleanedWithTypesCorrected[lendingCaseStudyDataFrameCleanedWithTypesCorrected['earliest_cr_line'] > lendingCaseStudyDataFrameCleanedWithTypesCorrected['issue_d']]
print(f"4. Number of inconsistent rows for earliest_cr_inconsistency: {len(earliest_cr_inconsistency)}")
print(earliest_cr_inconsistency[['earliest_cr_line', 'issue_d']])
print('---cleanup started---')
# removing the rows where this does not hold true.
lendingCaseStudyDataFrameCleanedWithTypesCorrected = lendingCaseStudyDataFrameCleanedWithTypesCorrected[~(lendingCaseStudyDataFrameCleanedWithTypesCorrected['earliest_cr_line'] > lendingCaseStudyDataFrameCleanedWithTypesCorrected['issue_d'])]
# checking incosistency again
earliest_cr_inconsistency = lendingCaseStudyDataFrameCleanedWithTypesCorrected[lendingCaseStudyDataFrameCleanedWithTypesCorrected['earliest_cr_line'] > lendingCaseStudyDataFrameCleanedWithTypesCorrected['issue_d']]
print(f"4.1 Updated Number of inconsistent rows for earliest_cr_inconsistency: {len(earliest_cr_inconsistency)}")
print(f"4.2 Number of rows after cleanup of earliest_cr_inconsistency: {len(lendingCaseStudyDataFrameCleanedWithTypesCorrected)}")
print()
# 5. Validate that open_acc <= total_acc
acc_inconsistency = lendingCaseStudyDataFrameCleanedWithTypesCorrected[lendingCaseStudyDataFrameCleanedWithTypesCorrected['open_acc'] > lendingCaseStudyDataFrameCleanedWithTypesCorrected['total_acc']]
print(f"5. Number of inconsistent rows for acc_inconsistency: {len(acc_inconsistency)}")
print()

# 6. Validate that total_rec_pricpal <= loan_amnt
# taking a difd, abs and comparing with 0.2 to avoid precision and rounding issues
prnc_inconsistency = lendingCaseStudyDataFrameCleanedWithTypesCorrected[(- lendingCaseStudyDataFrameCleanedWithTypesCorrected['total_rec_prncp'] + lendingCaseStudyDataFrameCleanedWithTypesCorrected['loan_amnt']) < -0.5]
print(f"6. Number of inconsistent rows for prnc_inconsistency: {len(prnc_inconsistency)}")
print()

# 8. Validate that pub_rec > pub_rec_bankruptcies
pubc_rec_inconsistency = lendingCaseStudyDataFrameCleanedWithTypesCorrected[lendingCaseStudyDataFrameCleanedWithTypesCorrected['pub_rec'] < lendingCaseStudyDataFrameCleanedWithTypesCorrected['pub_rec_bankruptcies']]
print(f"8. Number of inconsistent rows for pubc_rec_inconsistency: {len(pubc_rec_inconsistency)}")
print()

# Uncomment after debugging 

# 9. Validate Total Payments Consistency with interest and principal:
# 9.1 total_pymnt + total_pymnt_inv should be approximately equal to total_rec_prncp + total_rec_int + total_rec_late_fee + recoveries
# payments_rec_inconsistency1 = lendingCaseStudyDataFrameCleanedWithTypesCorrected[
#     abs(
#         (lendingCaseStudyDataFrameCleanedWithTypesCorrected['total_pymnt'] + lendingCaseStudyDataFrameCleanedWithTypesCorrected['total_pymnt_inv']) - 
#         (lendingCaseStudyDataFrameCleanedWithTypesCorrected['total_rec_prncp'] + lendingCaseStudyDataFrameCleanedWithTypesCorrected['total_rec_int'] + 
#          lendingCaseStudyDataFrameCleanedWithTypesCorrected['total_rec_late_fee'] + lendingCaseStudyDataFrameCleanedWithTypesCorrected['recoveries'])
#     ) > 1
# ]
# # 9.2 total_rec_prncp should be approximately equal to out_prncp + out_prncp_inv
# payments_rec_inconsistency2 = lendingCaseStudyDataFrameCleanedWithTypesCorrected[
#     abs(
#         lendingCaseStudyDataFrameCleanedWithTypesCorrected['total_rec_prncp'] - 
#         (lendingCaseStudyDataFrameCleanedWithTypesCorrected['out_prncp'] + lendingCaseStudyDataFrameCleanedWithTypesCorrected['out_prncp_inv'])
#     ) > 1
# ]
# print(abs(
#         lendingCaseStudyDataFrameCleanedWithTypesCorrected['total_rec_prncp'] - 
#         (lendingCaseStudyDataFrameCleanedWithTypesCorrected['out_prncp'] + lendingCaseStudyDataFrameCleanedWithTypesCorrected['out_prncp_inv'])
#     ))
# print(f"Number of inconsistent rows for payments_rec_inconsistency1: {len(payments_rec_inconsistency1)}")
# print(f"Number of inconsistent rows for payments_rec_inconsistency2: {len(payments_rec_inconsistency2)}")


# Establish relationship between percentage of funded amount by investors to loan status ??

# establish relationshuo between 



1.1 Number of inconsistent rows for loan amount vs funded_amnt: 0
1.2 Number of inconsistent rows for loan amount vs funded_amnt_inv: 0

2. Number of inconsistent rows for term_inconsistency: 0

3. Number of inconsistent rows for payment_d_inconsistency: 0

4. Number of inconsistent rows for earliest_cr_inconsistency: 0
Empty DataFrame
Columns: [earliest_cr_line, issue_d]
Index: []
---cleanup started---
4.1 Updated Number of inconsistent rows for earliest_cr_inconsistency: 0
4.2 Number of rows after cleanup of earliest_cr_inconsistency: 38881

5. Number of inconsistent rows for acc_inconsistency: 0

6. Number of inconsistent rows for prnc_inconsistency: 0

8. Number of inconsistent rows for pubc_rec_inconsistency: 0

